# Panda Example

In [5]:
import sys
sys.path.append('lib')

import scipy
import matplotlib.pyplot as plt
import numpy as np
import time, os
from IPython.display import clear_output

import pybullet as p
import pybullet_data

from pb_utils import set_q, vis_traj, create_primitives, get_joint_limits
from motion_planning import sampler, col_checker, interpolator, RRT

from functools import partial

DATA_PATH = '../../data'
robot_urdf = '/urdf/panda_arm.urdf'

%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup Pybullet

In [7]:
physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

In [29]:
p.resetSimulation()

### Setup Robot & environment

In [30]:
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)

plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,-.5), (0,0,0,1))

table_square_id = p.loadURDF('table_square/table_square.urdf')
p.resetBasePositionAndOrientation(table_square_id, (0.,0,-0.64), (0, 0, 0.7071068, 0.7071068))

table_id = p.loadURDF('table/table.urdf')
p.resetBasePositionAndOrientation(table_id, (.7,0,-0.5), (0, 0, 0.7071068, 0.7071068))

In [31]:
shelf_urdf = 'urdf/bookshelf_simple_collision.urdf'
shelf_id = p.loadURDF(fileName=shelf_urdf)
p.resetBasePositionAndOrientation(shelf_id, (-0.6,0.6,-0.5), (0, 0, 0, 1.))

#for visualizing the desired target
_,_,ball_id = create_primitives(radius=0.05)

In [32]:
set_q_std = partial(set_q,robot_id=robot_id, joint_indices=pb_joint_indices)

***

### Define the RRT

In [33]:
rob_simple_sampler = sampler(joint_limits)
rob_col_checker = col_checker(robot_id, pb_joint_indices, [ plane_id, shelf_id, table_id, table_square_id])
rob_interpolator = interpolator()

standard_rrt = RRT(7, rob_simple_sampler, rob_col_checker, rob_interpolator)

### Part 1: Planning from a random init config to a random goal config

#### Sample random init config

In [72]:
col_status = True
while col_status is True:
    q_init = rob_simple_sampler.sample().flatten()
    set_q_std(q_init)
    col_status = rob_col_checker.check_collision(q_init)

q_init adalah posisi awal robot. Posisi awal ditentukan secara random, tetapi berdasarkan joint limits yang ditentukan. Pada bagian ini, joint limits tidak ditentukan.<br>
q_goal adalah posisi akhir robot. Posisi akhir ditentukan secara random juga. 

#### Sample random goal config

In [73]:
col_status = True
while col_status is True:
    q_goal = rob_simple_sampler.sample().flatten()
    set_q_std(q_goal)
    col_status = rob_col_checker.check_collision(q_goal)
    

In [74]:
print(q_init)
print(q_goal)

[-1.6989 -0.4963 -0.6302 -3.0104 -2.7154  0.7485 -1.6761]
[ 0.0752 -0.0358 -2.8425 -1.938   2.7343  1.0954 -2.3567]


In [24]:
path = standard_rrt.plan(np.array(q_init), np.array(q_goal))

Trying to reach the goal state...
[array([ 2.2629,  0.6298,  2.0977, -2.2501,  1.2192,  1.3782, -0.9781]), array([ 2.3169,  0.6727,  2.1237, -2.2403,  1.2126,  1.4036, -1.0337])]
[ 2.3169  0.6727  2.1237 -2.2403  1.2126  1.4036 -1.0337]
Solution is found!
Planning...


In [25]:
vis_traj(path, set_q_std, dt = 0.1)

### Part 2: Planning from an initial EE position to a goal EE position

#### Get init config using IK

In [61]:
col_status = True
while col_status is True:
    set_q_std(rob_simple_sampler.sample().flatten())
    q_init = np.array(p.calculateInverseKinematics(robot_id, 
                                                   7,  
                                                   (0., 0.5, 0.55)))
    set_q_std(q_init)
    col_status = rob_col_checker.check_collision(q_init)

In [62]:
print(q_init)

[ 2.6673  0.7299 -2.1853 -2.2479 -2.6955  2.4893  0.4155]


#### Get goal config using IK

In [63]:
col_status = True
while col_status is True:
    set_q_std(rob_simple_sampler.sample().flatten())
    q_goal = np.array(p.calculateInverseKinematics(robot_id, 7,  (0., 0.5, 0.7)))
    set_q_std(q_goal)
    col_status = rob_col_checker.check_collision(q_goal)

In [64]:
print(q_goal)
# p.getEulerFromQuaternion(q_goal)

[-3.5665 -0.5949 -2.0849 -1.7756  2.363   1.0254  1.5788]


#### Plan

In [40]:
path = standard_rrt.plan(q_init, q_goal)

Trying to reach the goal state...
[array([-1.4284, -1.3714,  3.0424, -2.1101, -2.9898,  0.8305,  2.0637]), array([-1.468 , -1.4036,  3.0661, -2.1114, -3.0632,  0.8036,  2.0607])]
[-1.468  -1.4036  3.0661 -2.1114 -3.0632  0.8036  2.0607]
Solution is found!
Planning...


In [42]:
print(path)

[[ 0.4103  0.2978  2.0535 -2.02    0.6807  2.2238  2.3741]
 [ 0.4437  0.2681  1.995  -2.0289  0.656   2.1985  2.3199]
 [ 0.4771  0.2383  1.9366 -2.0377  0.6314  2.1732  2.2657]
 [ 0.5105  0.2086  1.8782 -2.0466  0.6067  2.1479  2.2115]
 [ 0.4709  0.1763  1.902  -2.0479  0.5333  2.121   2.2085]
 [ 0.4313  0.1441  1.9257 -2.0492  0.4599  2.0942  2.2054]
 [ 0.3918  0.1118  1.9495 -2.0505  0.3865  2.0673  2.2024]
 [ 0.3522  0.0796  1.9732 -2.0518  0.3131  2.0404  2.1994]
 [ 0.3126  0.0473  1.997  -2.0531  0.2397  2.0135  2.1964]
 [ 0.2731  0.0151  2.0208 -2.0543  0.1663  1.9866  2.1934]
 [ 0.2335 -0.0171  2.0445 -2.0556  0.0929  1.9597  2.1904]
 [ 0.1939 -0.0494  2.0683 -2.0569  0.0195  1.9328  2.1874]
 [ 0.1543 -0.0816  2.092  -2.0582 -0.0539  1.9059  2.1843]
 [ 0.1148 -0.1139  2.1158 -2.0595 -0.1273  1.8791  2.1813]
 [ 0.0752 -0.1461  2.1395 -2.0608 -0.2007  1.8522  2.1783]
 [ 0.0356 -0.1784  2.1633 -2.0621 -0.2741  1.8253  2.1753]
 [-0.0039 -0.2106  2.1871 -2.0634 -0.3475  1.7984  2.172

In [41]:
vis_traj(path, set_q_std)

In [6]:
p.disconnect()